## Import des librairies


In [21]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install category_encoders

In [22]:
tf.__version__

'2.3.0'

# Importing Dataset

In [23]:
dataset = pd.read_csv('export.csv')
x = dataset.iloc[:, [True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]].values # Toutes les colonnes à partir de l'index 3 jusqu'à l'avant dernière 
y = dataset.iloc[:, 5].values # Uniquement la dernière colonne du dataset (expected)

In [24]:
print(x[0])

['Guingamp' 'Monaco' 'French Ligue 1' '2016/17' 1 1 0 0 0 0 0 0 0 0 0 0 3
 2 2 0 0 0 0 0 0 0 0 0 0 3]


In [25]:
print(y)

['HOME' 'HOME' 'AWAY' ... 'AWAY' 'HOME' 'HOME']


# Encoding Data

In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import category_encoders as ce

ct = ColumnTransformer(transformers=[('encoder', ce.BinaryEncoder(), [0, 1, 2, 3])], remainder='passthrough')
cty = ColumnTransformer(transformers=[('encoder', ce.BinaryEncoder(), [0])], remainder='passthrough')

x = np.array(ct.fit_transform(x))

y=y.reshape(-1,1)

y = np.array(cty.fit_transform(y))


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [27]:
print(y)

[[0 0 1]
 [0 0 1]
 [0 1 0]
 ...
 [0 1 0]
 [0 0 1]
 [0 0 1]]


In [28]:
print(x)

[[0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 ...
 [0 0 0 ... 6 4 7]
 [0 0 0 ... 6 1 6]
 [0 0 1 ... 1 2 7]]


# Splitting the dataset into the Training set and Test set


In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [30]:
 from sklearn.preprocessing import StandardScaler
 sc = StandardScaler()
 x_train = sc.fit_transform(x_train)
 x_test = sc.transform(x_test)

# Init ANN

In [92]:
ann = tf.keras.models.Sequential()

In [93]:
input_dimension = x_test[0].shape[0]
print(input_dimension)

52


### Add layers


In [94]:
ann.add(tf.keras.layers.Dense(30, activation='relu', input_dim=input_dimension))

### Output Layers

In [95]:
ann.add(tf.keras.layers.Dense(units = 3, activation='softmax')) 

### Compile

In [96]:
from keras.metrics import categorical_accuracy

ann.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = [categorical_accuracy])

### Training

In [97]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 20, validation_data=(x_test, y_test))

Epoch 1/20
231/231 [==============================] - 0s 2ms/step - loss: 0.7793 - categorical_accuracy: 0.6119 - val_loss: 0.6697 - val_categorical_accuracy: 0.6363
Epoch 2/20
231/231 [==============================] - 0s 1ms/step - loss: 0.6547 - categorical_accuracy: 0.6519 - val_loss: 0.6548 - val_categorical_accuracy: 0.6618
Epoch 3/20
231/231 [==============================] - 0s 1ms/step - loss: 0.6406 - categorical_accuracy: 0.6585 - val_loss: 0.6423 - val_categorical_accuracy: 0.6591
Epoch 4/20
231/231 [==============================] - 0s 1ms/step - loss: 0.6337 - categorical_accuracy: 0.6589 - val_loss: 0.6513 - val_categorical_accuracy: 0.6434
Epoch 5/20
231/231 [==============================] - 0s 1ms/step - loss: 0.6305 - categorical_accuracy: 0.6671 - val_loss: 0.6372 - val_categorical_accuracy: 0.6493
Epoch 6/20
231/231 [==============================] - 0s 1ms/step - loss: 0.6288 - categorical_accuracy: 0.6607 - val_loss: 0.6381 - val_categorical_accuracy: 0.6585
Epoc

### Predict

In [98]:
pred_train= ann.predict(x_train)
scores = ann.evaluate(x_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= ann.predict(x_test)
scores2 = ann.evaluate(x_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))   

Accuracy on training data: 0.6636856198310852% 
 Error on training data: 0.3363143801689148
Accuracy on test data: 0.649322509765625% 
 Error on test data: 0.350677490234375
